In [1]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import wandb
import copy

In [2]:
import torch
torch.cuda.empty_cache()

In [4]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'psudo_cascade_swin_t'
# config file 들고오기
cfg = Config.fromfile('./_cascade_rcnn/cascade_swin_t_fpn_3x_coco.py')
root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.classes = classes

cfg.data.train.img_prefix = root
cfg.data.train.ann_file = './upsampling_stratified_kfold/cv_train_1.json' 

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = './upsampling_stratified_kfold/cv_val_1.json' 

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'

cfg.data.samples_per_gpu = 6

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/' + model_name

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)

In [5]:
# build_dataset
# datasets = [build_dataset(cfg.data.train),build_dataset(cfg.data.val)]
datasets = [build_dataset(cfg.data.train)]
datasets

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!


[
 CocoDataset Train dataset with number of images 8762, and instance counts: 
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | category          | count | category      | count | category        | count | category    | count | category     | count |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | 0 [General trash] | 3890  | 1 [Paper]     | 6081  | 2 [Paper pack]  | 1048  | 3 [Metal]   | 1076  | 4 [Glass]    | 935   |
 | 5 [Plastic]       | 3255  | 6 [Styrofoam] | 1348  | 7 [Plastic bag] | 6732  | 8 [Battery] | 189   | 9 [Clothing] | 736   |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [7]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-03-31 19:37:21,530 - mmdet - INFO - Start running, host: root@6f46492e33de, work_dir: /opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/psudo_cascade_swin_t
2022-03-31 19:37:21,532 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(LOW        

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:222: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')
wandb: Currently logged in as: ysw2946 (use `wandb login --relogin` to force relogin)


AssertionError: Caught AssertionError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/ml/detection/baseline/mmdetection/mmdet/datasets/custom.py", line 218, in __getitem__
    data = self.prepare_train_img(idx)
  File "/opt/ml/detection/baseline/mmdetection/mmdet/datasets/custom.py", line 241, in prepare_train_img
    return self.pipeline(results)
  File "/opt/ml/detection/baseline/mmdetection/mmdet/datasets/pipelines/compose.py", lin

In [ ]:
import torch
torch.cuda.empty_cache()